# Specialist Search Agent (SSA) – RAG Pipeline (V3)

This notebook implements a production-ready Referral Matching Agent using an advanced agentic RAG approach:
- Zero-shot LLM-based specialty extraction (BART-MNLI)
- FAISS-based semantic retrieval over specialist profiles
- Composite scoring with geolocation-based proximity, insurance, and availability
- GPT-2–based explanation generation for patient-friendly recommendations
- Modular multi-agent pipeline with Supervisor and Insurance Authorization agents
- Reflection loop for low-confidence fallback queries


# Import Libraries

In [2]:
!pip install langchain faiss-cpu sentence-transformers -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [1]:
import pandas as pd
import json
import random

# Import FAISS vector store and embeddings from LangChain.
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Import Hugging Face pipelines for zero-shot classification and text generation.
from transformers import pipeline

# Import geopy for geolocation calculations.
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [14]:
# Initialize geopy's geolocator.
geolocator = Nominatim(user_agent="specialist_locator")

# Initialize the advanced medical intent classifier using zero-shot classification.
# Candidate specialties can be expanded as needed.
candidate_specialties = ["Cardiologist", "Pulmonologist", "Neurologist", "General Practitioner", "Orthopedic", "Dermatologist"]
intent_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Initialize text generation pipeline to enhance explanations.
explanation_generator = pipeline("text-generation", model="gpt2", pad_token_id=50256)


Device set to use cpu
Device set to use cpu


# Data Loading

In [3]:
# Load provider metadata CSV.
df = pd.read_csv("Mock_Specialist_Dataset.csv")
print("Provider Metadata Sample:")
print(df.head())
print("CSV Columns:")
print(df.columns)

Provider Metadata Sample:
   specialist_id                 name          specialty        location  \
0              1      Dr. Aarti Mehta       Cardiologist   New Haven, CT   
1              2     Dr. James Wright      Pulmonologist    Stamford, CT   
2              3     Dr. Leena Kapoor        Neurologist    Hartford, CT   
3              4       Dr. Brian Choi  Internal Medicine  Bridgeport, CT   
4              5  Dr. Sophia Martinez       Cardiologist     Norwalk, CT   

  insurance_accepted  availability_next_3_days  \
0   BlueCross, Aetna                         3   
1       Aetna, Cigna                         0   
2  United, BlueCross                         2   
3    Medicare, Aetna                         5   
4  BlueCross, United                         1   

                                     profile_summary  
0  Board-certified cardiologist with 10 years of ...  
1  Specialist in respiratory and lung disorders i...  
2  Neurodiagnostics expert focusing on dizziness,..

# Data preparation

In [4]:
# Prepare document texts and metadata for each provider.
texts = []
metadatas = []
for idx, row in df.iterrows():
    doc_text = (
        f"Specialist profile for {row['name']}. "
        f"Specialty: {row['specialty']}. "
        f"Location: {row['location']}. "
        f"Insurance accepted: {row['insurance_accepted']}. "
        f"Availability next 3 days: {row['availability_next_3_days']}. "
        f"Profile summary: {row['profile_summary']}"
    )
    texts.append(doc_text)
    metadatas.append(row.to_dict())

In [5]:
texts

['Specialist profile for Dr. Aarti Mehta. Specialty: Cardiologist. Location: New Haven, CT. Insurance accepted: BlueCross, Aetna. Availability next 3 days: 3. Profile summary: Board-certified cardiologist with 10 years of experience in treating arrhythmia and heart failure.',
 'Specialist profile for Dr. James Wright. Specialty: Pulmonologist. Location: Stamford, CT. Insurance accepted: Aetna, Cigna. Availability next 3 days: 0. Profile summary: Specialist in respiratory and lung disorders including COPD and asthma. 15+ years in pulmonary care.',
 'Specialist profile for Dr. Leena Kapoor. Specialty: Neurologist. Location: Hartford, CT. Insurance accepted: United, BlueCross. Availability next 3 days: 2. Profile summary: Neurodiagnostics expert focusing on dizziness, migraines, and stroke recovery.',
 'Specialist profile for Dr. Brian Choi. Specialty: Internal Medicine. Location: Bridgeport, CT. Insurance accepted: Medicare, Aetna. Availability next 3 days: 5. Profile summary: General pr

In [6]:
metadatas

[{'specialist_id': 1,
  'name': 'Dr. Aarti Mehta',
  'specialty': 'Cardiologist',
  'location': 'New Haven, CT',
  'insurance_accepted': 'BlueCross, Aetna',
  'availability_next_3_days': 3,
  'profile_summary': 'Board-certified cardiologist with 10 years of experience in treating arrhythmia and heart failure.'},
 {'specialist_id': 2,
  'name': 'Dr. James Wright',
  'specialty': 'Pulmonologist',
  'location': 'Stamford, CT',
  'insurance_accepted': 'Aetna, Cigna',
  'availability_next_3_days': 0,
  'profile_summary': 'Specialist in respiratory and lung disorders including COPD and asthma. 15+ years in pulmonary care.'},
 {'specialist_id': 3,
  'name': 'Dr. Leena Kapoor',
  'specialty': 'Neurologist',
  'location': 'Hartford, CT',
  'insurance_accepted': 'United, BlueCross',
  'availability_next_3_days': 2,
  'profile_summary': 'Neurodiagnostics expert focusing on dizziness, migraines, and stroke recovery.'},
 {'specialist_id': 4,
  'name': 'Dr. Brian Choi',
  'specialty': 'Internal Medi

# Vector embedding

In [7]:
# Initialize Hugging Face Sentence Transformer for embeddings.
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build the FAISS vector store.
vector_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
print("Vector store created successfully.")

<ipython-input-7-54520e101a74>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully.


# Vector embedding

In [15]:
# Advanced Medical Intent Extraction using zero-shot classification.
def advanced_extract_specialty(patient_query: str) -> str:
    hypothesis = "The patient might need a {}."
    result = intent_classifier(patient_query, candidate_labels=candidate_specialties, hypothesis=hypothesis)
    # Return the label with the highest score.
    return result["labels"][0]

from functools import lru_cache

@lru_cache(maxsize=128)
def get_coordinates(location_str: str):
    try:
        location = geolocator.geocode(location_str)
        if location:
            return (location.latitude, location.longitude)
    except Exception as e:
        print(f"Geocoding error for {location_str}: {e}")
    return None

# Geolocation-Based Proximity Scoring.
def calculate_proximity_score(provider_location: str, patient_location: str):
    provider_coords = get_coordinates(provider_location)
    patient_coords = get_coordinates(patient_location)
    if provider_coords and patient_coords:
        dist = geodesic(provider_coords, patient_coords).km
        # Define scoring based on distance (example thresholds):
        if dist <= 10:
            return 1, dist
        elif dist <= 30:
            score = 1 - ((dist - 10) / 20)  # Linear interpolation between 10 km and 30 km.
            return score, dist
        else:
            return 0, dist
    return 0, None

# Simulated real-time update to provider metadata.
def update_provider_metadata(metadata: dict):
    # For demonstration, randomly adjust availability.
    metadata["availability_next_3_days"] = random.choice([0, 1, 2, 3, 4, 5])
    # In a real system, you would call an external API to refresh this data.
    return metadata

# Enhanced explanation using a text generation model.
def generate_explanation(details: dict, candidate_metadata: dict):
    input_prompt = (
        f"Explain why this doctor is a good match for the patient. "
        f"Score details - Specialty: {details['specialty_score']}, "
        f"Insurance: {details['insurance_score']}, "
        f"Availability: {details['availability_score']:.2f}, "
        f"Proximity: {details['proximity_score']:.2f}. "
        f"Doctor's profile: {candidate_metadata['profile_summary']}"
    )
    # Use max_new_tokens to generate additional tokens beyond the input prompt.
    explanation = explanation_generator(
        input_prompt,
        max_new_tokens=20,
        truncation=True,
        do_sample=False
    )[0]['generated_text']
    return explanation.strip()

# Modular Multi-Agent Pipeline: Define downstream agents.
def referral_supervisor_agent(ssa_output: dict):
    """
    Simulates additional checks by a Referral Supervisor Agent.
    For each candidate, adds a supervisor review decision.
    """
    for candidate in ssa_output["specialist_matches"]:
        candidate["supervisor_review"] = "Approved" if candidate["composite_score"] >= 0.7 else "Needs Review"
    return ssa_output

def insurance_authorization_agent(ssa_output: dict):
    """
    Simulates insurance authorization by verifying candidate details.
    For each candidate, adds an insurance authorization status.
    """
    for candidate in ssa_output["specialist_matches"]:
        candidate["insurance_auth_status"] = "Pre-Authorized" if candidate["composite_score"] >= 0.8 else "Pending"
    return ssa_output

# RAG Pipeline

In [16]:
# Upgraded RAG Pipeline Function using all enhancements.
def rag_pipeline_advanced(patient_query: str,
                          vector_store: FAISS,
                          patient_location="New Haven, CT",
                          patient_insurance="BlueCross",
                          confidence_threshold: float = 0.6):
    # Step 1: Advanced extraction of recommended specialty.
    recommended_specialty = advanced_extract_specialty(patient_query)

    # Step 2: Semantic search using a query based on recommended specialty.
    search_query = f"Specialty: {recommended_specialty}"
    search_results = vector_store.similarity_search(search_query, k=5)

    candidates = []
    for doc in search_results:
        # Simulate real-time data update.
        updated_metadata = update_provider_metadata(doc.metadata)
        # Specialty score.
        specialty_score = 1 if updated_metadata.get("specialty", "").strip().lower() == recommended_specialty.lower() else 0
        # Insurance score.
        insurances = [x.strip().lower() for x in updated_metadata.get("insurance_accepted", "").split(',')]
        insurance_score = 1 if patient_insurance.lower() in insurances else 0
        # Availability score.
        availability = updated_metadata.get("availability_next_3_days", 0)
        availability_score = 0 if availability <= 0 else max(0, (10 - availability) / 10)
        # Geolocation-based proximity score.
        proximity_score, distance_value = calculate_proximity_score(updated_metadata.get("location", ""), patient_location)
        # Composite score calculation with weights.
        composite = (0.4 * specialty_score + 0.2 * insurance_score + 0.2 * availability_score + 0.2 * proximity_score)
        details = {"specialty_score": specialty_score,
                   "insurance_score": insurance_score,
                   "availability_score": availability_score,
                   "proximity_score": proximity_score}
        # Enhanced explanation.
        explanation = generate_explanation(details, updated_metadata)

        candidate = {
            "name": updated_metadata.get("name", "Unknown"),
            "specialty": updated_metadata.get("specialty", "Unknown"),
            "composite_score": round(composite, 2),
            "explanation": explanation,
            "metadata": updated_metadata,
            "distance_km": distance_value
        }
        candidates.append(candidate)

    # Sort by composite score (highest first) and select top 3.
    candidates = sorted(candidates, key=lambda x: x["composite_score"], reverse=True)
    top_candidates = candidates[:3]

    # Reflection loop: if top scores are too low, re-query with a refined prompt.
    if all(c["composite_score"] < confidence_threshold for c in top_candidates):
        refined_query = f"Expert in {recommended_specialty} with high availability and network compatibility"
        search_results = vector_store.similarity_search(refined_query, k=5)
        candidates = []
        for doc in search_results:
            updated_metadata = update_provider_metadata(doc.metadata)
            specialty_score = 1 if updated_metadata.get("specialty", "").strip().lower() == recommended_specialty.lower() else 0
            insurances = [x.strip().lower() for x in updated_metadata.get("insurance_accepted", "").split(',')]
            insurance_score = 1 if patient_insurance.lower() in insurances else 0
            availability = updated_metadata.get("availability_next_3_days", 0)
            availability_score = 0 if availability <= 0 else max(0, (10 - availability) / 10)
            proximity_score, distance_value = calculate_proximity_score(updated_metadata.get("location", ""), patient_location)
            composite = (0.4 * specialty_score + 0.2 * insurance_score + 0.2 * availability_score + 0.2 * proximity_score)
            details = {"specialty_score": specialty_score,
                       "insurance_score": insurance_score,
                       "availability_score": availability_score,
                       "proximity_score": proximity_score}
            explanation = generate_explanation(details, updated_metadata)
            candidate = {
                "name": updated_metadata.get("name", "Unknown"),
                "specialty": updated_metadata.get("specialty", "Unknown"),
                "composite_score": round(composite, 2),
                "explanation": explanation,
                "metadata": updated_metadata,
                "distance_km": distance_value
            }
            candidates.append(candidate)
        candidates = sorted(candidates, key=lambda x: x["composite_score"], reverse=True)
        top_candidates = candidates[:3]

    output = {
         "patient_query": patient_query,
         "recommended_specialty": recommended_specialty,
         "patient_location": patient_location,
         "patient_insurance": patient_insurance,
         "specialist_matches": top_candidates
    }
    return output


# Final Call

In [17]:
# Sample patient free-text input.
patient_query = "I'm having chest pain during mild activity, it gets worse with exertion, and sometimes I feel dizzy."

# Run the advanced RAG pipeline.
ssa_output = rag_pipeline_advanced(patient_query, vector_store)
print("SSA Output:")
print(json.dumps(ssa_output, indent=4))

# Modular Multi-Agent Pipeline: Pass the SSA output to downstream agents.
ssa_with_supervisor = referral_supervisor_agent(ssa_output)
final_output = insurance_authorization_agent(ssa_with_supervisor)

print("\nFinal Multi-Agent Pipeline Output:")
print(json.dumps(final_output, indent=4))

SSA Output:
{
    "patient_query": "I'm having chest pain during mild activity, it gets worse with exertion, and sometimes I feel dizzy.",
    "recommended_specialty": "Pulmonologist",
    "patient_location": "New Haven, CT",
    "patient_insurance": "BlueCross",
    "specialist_matches": [
        {
            "name": "Dr. Aarti Mehta",
            "specialty": "Cardiologist",
            "composite_score": 0.52,
            "explanation": "Explain why this doctor is a good match for the patient. Score details - Specialty: 0, Insurance: 1, Availability: 0.60, Proximity: 1.00. Doctor's profile: Board-certified cardiologist with 10 years of experience in treating arrhythmia and heart failure.\n\nDr. Robert J. K. Koppel, MD, is a cardiologist and",
            "metadata": {
                "specialist_id": 1,
                "name": "Dr. Aarti Mehta",
                "specialty": "Cardiologist",
                "location": "New Haven, CT",
                "insurance_accepted": "BlueCros

In [19]:
import os

output_path = "ssa_final_output_v3.json"
with open(output_path, "w") as f:
    json.dump(final_output, f, indent=4)

print(f"\nOutput saved to: {os.path.abspath(output_path)}")


Output saved to: /content/ssa_final_output_v3.json
